In [ ]:
import WDFunctions as w
import glob

year_of_data = '2021'

location = '*' # path name to directory of files, * for current directory
files = glob.glob('*')

file = [file for file in files if 'published' in file.lower()][0] #'publishedweek202020.xlsx'
tabs = w.loadxlstabs(file)
w.WeeklyDeathsByRegion(tabs)
w.WeeklyDeathsByAgeSex(tabs)

file = [file for file in files if 'lahb' in file] # 'lahbtablesweek20finalcodes.xlsx'
if len(file) == 0: 
    file = [file for file in files if 'la_hb' in file.lower()][0]
else: file = file[0]
reg_data = w.pd.read_excel(file, sheet_name='Registrations - All data', skiprows=3)
occ_data = w.pd.read_excel(file, sheet_name='Occurrences - All data', skiprows=3)
w.WeeklyDeathsByLA_HB(reg_data, occ_data, year_of_data)

print('Transforms complete!')
